## Etapa 1: Instalação das bibliotecas

In [0]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 58kB/s 
     |████████████████████████████████| 3.2MB 32.7MB/s 
     |████████████████████████████████| 368kB 42.9MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0


## Etapa 2: Importação das bibliotecas

In [0]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'1.13.1'

## Etapa 3: Pré-processamento

### Carregando a base de dados FashionMNIST

In [0]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Normalização das imagens

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

### Mudando a dimensionalidade da base de dados

In [0]:
X_train.shape

(60000, 28, 28)

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [0]:
X_train.shape

(60000, 784)

## Etapa 4: Construindo o modelo

### Definindo o modelo

In [0]:
model = tf.keras.models.Sequential()

### Construindo o modelo

In [0]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilando o modelo

In [0]:
y_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Treinando o modelo

In [0]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 12s 198us/sample - loss: 1.3389 - sparse_categorical_accuracy: 0.4173
Epoch 2/5
60000/60000 [==============================] - 11s 182us/sample - loss: 0.7342 - sparse_categorical_accuracy: 0.6905
Epoch 3/5
60000/60000 [==============================] - 11s 185us/sample - loss: 0.5065 - sparse_categorical_accuracy: 0.8270
Epoch 4/5
60000/60000 [==============================] - 11s 189us/sample - loss: 0.4513 - sparse_categorical_accuracy: 0.8443
Epoch 5/5
60000/60000 [==============================] - 11s 187us/sample - loss: 0.4211 - sparse_categorical_accuracy: 0.8569


### Avaliando o modelo

In [0]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 77us/sample - loss: 0.4108 - sparse_categorical_accuracy: 0.8569
Test accuracy: 0.8568999767303467


## Etapa 5: Convertendo o modelo para o TensorFlow Lite

### Salvando o modelo

In [0]:
model_name = "fashion_mobile_model.h5"
tf.keras.models.save_model(model, model_name)

### Criando o TFLite Converter

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

INFO:tensorflow:Froze 12 variables.
INFO:tensorflow:Converted 12 variables to const ops.


### Convertendo o modelo

In [0]:
tflite_model = converter.convert()

### Salvando a versão TFLite

In [0]:
with open("tflite_model", "wb") as f:
  f.write(tflite_model)